In [1]:
# Code to solve a Version of the Lukas 78 model with inattention
# A bit hacky at several points, but should work

using Pkg   
using Distributions,Plots,Parameters,LinearAlgebra,Optim,Roots
@with_kw struct model             
    ## Model parameters:
    β = 0.95 # discount factor
    σ = 2 # RRA/IES
    θ = 0.5 # probability of no information update
    w = 1.0 # fixed riskless income
    AS = 1.0 # size of money tree
   
    ## aggregate state:
    nz = 2 # number of agg states
    γgrid = [0.98,1.02] # grid for aggregate state
    p_bg = 0.2 # probability of bad -> good
    p_gg = 0.6 # probability of good -> good
    γprob = [1-p_bg p_bg ; 1-p_gg p_gg] # transition matrix for aggregate state, [x,y] gives probability of transition from x to y
    
    ## asset grid:
    na = 100 # number of asset grid points
    amin = 0.0 # borrowing constraint
    amax = 1.0 # maximum asset level
    agrid = range(amin,stop=amax,length=na) # asset grid

    ## periods since update grid:
    kmin = 0 # minimum periods since update
    kmax = 20 # θ^kmax should be very small
    nk = 20 # number of periods since update grid points
    kgrid = range(kmin,stop=kmax,length=nk+1) # periods since update grid

    ## collect all
    stateind = collect(Iterators.product(1:na, 1:nz, 1:nz, 1:nk)) # matrix of state indicies (assets, true z, last observed z, nk)

    ## build a k period forward transition matrix
    γprob_kfwd = [γprob^kgrid[ik+1] for ik in 1:nk]

    ## numerical parameters
    maxit = 20 # vfi maxiteration
    tol = 1e-3 # vfi tolerance
    Tburn = 100 # burn in periods
    TT = 500 # simulation length
    maxit_bis = 10 # maximum number of iterations for bisection
    tol_bis = 1e-3 # tolerance for bisection
end

# utility function
function u(par::model,c)
    @unpack σ = par
    if c <= 0
        return -9999999999
    end
    if σ == 1
        return log(c)
    else
        return c^(1-σ)/(1-σ)
    end
end;

include("VFI.jl");
include("aggregates.jl");




(process:15100): GLib-GIO-WARNING **: 19:50:51.699: Unexpectedly, UWP app `Microsoft.ScreenSketch_11.2312.33.0_x64__8wekyb3d8bbwe' (AUMId `Microsoft.ScreenSketch_8wekyb3d8bbwe!App') supports 29 extensions but has no verbs

(process:15100): GLib-GIO-WARNING **: 19:50:51.732: Unexpectedly, UWP app `Clipchamp.Clipchamp_2.9.1.0_neutral__yxz26nhyzhsrt' (AUMId `Clipchamp.Clipchamp_yxz26nhyzhsrt!App') supports 41 extensions but has no verbs

(process:15100): GLib-GIO-WARNING **: 19:50:51.839: Unexpectedly, UWP app `Microsoft.OutlookForWindows_1.2024.207.0_x64__8wekyb3d8bbwe' (AUMId `Microsoft.OutlookForWindows_8wekyb3d8bbwe!Microsoft.OutlookforWindows') supports 4 extensions but has no verbs


LoadError: LoadError: syntax: missing comma or ) in argument list
in expression starting at c:\Users\jonas\OneDrive\Economics\Github\Lukas_INAT\src\aggregates.jl:52

In [27]:
# guess pz's_
pzguess = [1.0,1.0];
# test vfi:
v,a = VFI(model(),pzguess);

In [24]:
# draw a shock series:
zshocks = drawz(model());
# simulate:
# jdT,ap,ADT = simulate(model(),pzguess,zshocks);

# test bisection:
pvec = p_finder(model(),pzguess.*2,pzguess.*0.5,zshocks);

 -- VFI done, Iteration: 19 Diff: diff


 -- VFI done, Iteration: 20 Diff: diff


Initial Bisection Boundaries not set correctly


 -- VFI done, Iteration: 20 Diff: diff

BoundsError: BoundsError: attempt to access 500-element Vector{Float64} at index [600-element BitVector]

In [19]:
# test the distribution transition fct:
jd_1 = jdguess(model());
jd_2 = jdtrans(model(),jd_1,apol,1); # 1 here = bad state today

600-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 2
 1
 1
 2
 ⋮
 2
 2
 2
 1
 1
 1
 1
 1
 1

In [21]:
xx = zshocks .* 5

600-element Vector{Int64}:
  5
  5
  5
  5
  5
  5
 10
  5
  5
 10
  ⋮
 10
 10
 10
  5
  5
  5
  5
  5
  5

In [22]:
xx[zshocks .== 1]

413-element Vector{Int64}:
 5
 5
 5
 5
 5
 5
 5
 5
 5
 5
 ⋮
 5
 5
 5
 5
 5
 5
 5
 5
 5